In [8]:
from osgeo import gdal
import numpy as np

In [6]:
def load_image(image):
    return gdal.Open(image, gdal.GA_ReadOnly)
    
def convert_to_array(dataset):
    bands = [dataset.GetRasterBand(i + 1).ReadAsArray() for i in range(dataset.RasterCount)]
    return np.stack(bands, 2)

def normalize(img):
    min = img.min()
    max = img.max()
    x = 2.0 * (img - min) / (max - min) - 1.0
    return x

def normalize_(img):
    min = img.min()
    max = img.max()
    x = (img - min) / (max - min)
    return x

#############################
### Custom Loss functions ###
#############################
# --------------------------------------------------------------------------------------
def dice_coef(y_true, y_pred, smooth = 1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def soft_dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

#Source: https://towardsdatascience.com/metrics-to-evaluate-your-semantic-segmentation-model-6bcb99639aa2
from keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)

    return iou
# --------------------------------------------------------------------------------------

def convert_to_raster(image, predicted, output):
    cols = image.RasterXSize
    rows = image.RasterYSize
    ds = image
    
    driver = gdal.GetDriverByName('GTiff')
    outDs = driver.Create(output, cols, rows, 1, gdal.GDT_Float32)
    outDs.GetRasterBand(1).WriteArray(predicted)

    # follow code is adding GeoTransform and Projection
    geotrans = ds.GetGeoTransform()  # get GeoTransform from existed 'data0'
    proj = ds.GetProjection()  # you can get from a existed tif or import
    outDs.SetGeoTransform(geotrans)
    outDs.SetProjection(proj)
    outDs.FlushCache()
    outDs = None